In [2]:
# Imports
from qiskit import *
import qiskit.quantum_info as qi
from qiskit_ibm_provider import IBMProvider
import numpy as np
import galois

In [3]:
provider = IBMProvider()

In [4]:
backend = provider.get_backend('simulator_mps')

## Simon's Function

In [5]:
def simon_func(s: str):
    # Create a QuantumCircuit implementing a query gate for simon problem
    # Obeying the promise, to find the hidden string 's'
    n = len(s)
    qc = QuantumCircuit(2 * n)
    pi = np.random.permutation(2**n)  # Creates essentially a random array, hiding the string

    query_gate = np.zeros((4**n,4**n))
    # Now we'll define a query gate explicitly. The idea is to first define a function g(x) = min{x,x ^ s}, which
    # is a simple function that satisfies the promise, and then we take f to be the composition of g and the random
    # permutation pi. This gives us a random function satisfying the promise for s.
    
    for x in range(2**n):
        for y in range(2**n):
            #print(x,y, int(s,2),x ^ int(s,2))
            z = y ^ pi[min(x, x ^ int(s,2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1  # Huh???
    qc.unitary(query_gate, range(2*n))
    return qc#, query_gate, s

In [6]:
# Setting up our specific circuit to run the operation(using compose)
def measurements(problem, k: int):
    # k is the number of times to confirm everything for post-processing
    # quantum circuit uses the Unitary function given to it
    n = problem.num_qubits // 2

    qc = QuantumCircuit(2 * n, n)
    qc.h(range(n))  # Hadamard gate to everything
    qc.compose(problem, inplace = True)  # Add the function to our circuit
    qc.h(range(n))
    qc.measure(range(n), range(n))

    result = backend.run(qc, shots = k, memory=True).result()
    return result.get_memory()

In [7]:
measurements(simon_func('10010'), k = 20)

['11011',
 '10110',
 '01100',
 '10111',
 '00100',
 '10110',
 '10110',
 '10011',
 '01101',
 '01000',
 '11010',
 '00001',
 '01000',
 '11011',
 '10110',
 '01000',
 '10011',
 '00000',
 '10111',
 '01101']

In [8]:
# Given a Quantum Circuit that implements a query gate, return the hidden string

# Quantum part: run the circuit defined proviously part k times and gather the measurement results
# Replace +10 by +r for any nonnegative integer r depending on desired confidence
def full_algo(problem):
    meas = measurements(problem, k = problem.num_qubits // 2 + 10)
    print("Measurements:")
    print(meas)

    # Classical post-processing
    
    # 1. Convert measurements to 2D-array of integers
    matrix = np.array([list(bitstring) for bitstring in meas]).astype(int)

    # 2. Interpret matrix as using arithmetic mod 2, and null space
    null_space = galois.GF(2)(matrix).null_space()
    print("Null space")
    display(null_space)

    # 3. Convert back to a string
    print('Guess for hidden string s:')
    if len(null_space) == 0:
        # No non-trivial solution: 's' is all zeroes
        return '0' * len(meas[0])
    return ''.join(np.array(null_space[0]).astype(str))

In [9]:
hidden_string = '110011'
through_func = simon_func(hidden_string)

In [10]:
full_algo(through_func)

Measurements:
['011010', '110011', '001011', '111111', '011010', '011110', '101010', '110100', '010001', '011010', '000000', '001111', '100110', '110111', '101010', '111000']
Null space


GF([[1, 1, 0, 0, 1, 1]], order=2)

Guess for hidden string s:


'110011'